In [28]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [6]:
df= pd.read_csv(r"D:\Projects\Diamond price\notebooks\diamonds.csv")
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [11]:
df.drop("Unnamed: 0", axis=1, inplace=True)


In [12]:
df.sample()

,carat,cut,color,clarity,depth,table,price,x,y,z
53899,0.72,Ideal,H,VVS2,62.3,56.0,2752,5.74,5.81,3.6


In [13]:
X= df.drop("price", axis=1)
y= df["price"]

In [14]:
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.20, random_state=42)
X_train

,carat,cut,color,clarity,depth,table,x,y,z
26546,2.01,Good,F,SI2,58.1,64.0,8.23,8.19,4.77
9159,1.01,Very Good,E,SI2,60.0,60.0,6.57,6.49,3.92
14131,1.10,Premium,H,VS2,62.5,58.0,6.59,6.54,4.10
15757,1.50,Good,E,SI2,61.5,65.0,7.21,7.17,4.42
24632,1.52,Very Good,G,VS1,62.1,57.0,7.27,7.32,4.53
...,...,...,...,...,...,...,...,...,...
11284,1.05,Very Good,I,VS2,62.4,59.0,6.48,6.51,4.05
44732,0.47,Ideal,D,VS1,61.0,55.0,5.03,5.01,3.06
38158,0.33,Very Good,F,IF,60.3,58.0,4.49,4.46,2.70
860,0.90,Premium,J,SI1,62.8,59.0,6.13,6.03,3.82


In [15]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [63]:
trf1= ColumnTransformer([
("OE", OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories]), [1,2,3 ] )
], remainder="passthrough")

In [57]:
trf2= ColumnTransformer([
    ("SS", StandardScaler(), ["carat", "depth",	"table",	"x",	"y",	"z"])
    ], remainder="passthrough" )

In [58]:
trf3= SelectKBest(score_func=chi2,k=7)

In [106]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [114]:
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet(),
'RandomForestRegressor': RandomForestRegressor(n_estimators=100,
criterion='squared_error',
max_samples= 0.75,
max_depth=12)
}

In [48]:
trained_model_list=[]
model_list=[]
r2_list=[]

In [49]:
list(models)

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet', 'RandomForestRegressor']

In [50]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    print(model)

LinearRegression()
Lasso()
Ridge()
ElasticNet()
RandomForestRegressor(criterion='gini', max_depth=12, max_features='auto',
                      max_samples=0.75)


In [51]:
models.keys()

dict_keys(['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet', 'RandomForestRegressor'])

In [52]:
models.values()

dict_values([LinearRegression(), Lasso(), Ridge(), ElasticNet(), RandomForestRegressor(criterion='gini', max_depth=12, max_features='auto',
                      max_samples=0.75)])

In [75]:
X_train

,carat,cut,color,clarity,depth,table,x,y,z
26546,2.01,Good,F,SI2,58.1,64.0,8.23,8.19,4.77
9159,1.01,Very Good,E,SI2,60.0,60.0,6.57,6.49,3.92
14131,1.10,Premium,H,VS2,62.5,58.0,6.59,6.54,4.10
15757,1.50,Good,E,SI2,61.5,65.0,7.21,7.17,4.42
24632,1.52,Very Good,G,VS1,62.1,57.0,7.27,7.32,4.53
...,...,...,...,...,...,...,...,...,...
11284,1.05,Very Good,I,VS2,62.4,59.0,6.48,6.51,4.05
44732,0.47,Ideal,D,VS1,61.0,55.0,5.03,5.01,3.06
38158,0.33,Very Good,F,IF,60.3,58.0,4.49,4.46,2.70
860,0.90,Premium,J,SI1,62.8,59.0,6.13,6.03,3.82


In [91]:
X.select_dtypes(include='object')

,cut,color,clarity
0,Ideal,E,SI2
1,Premium,E,SI1
2,Good,E,VS1
3,Premium,I,VS2
4,Good,J,SI2
...,...,...,...
53935,Ideal,D,SI1
53936,Good,D,SI1
53937,Very Good,D,SI1
53938,Premium,H,SI2


In [90]:
categorical_cols=X.select_dtypes(include='object').columns
numerical_cols=X.select_dtypes(exclude='object').columns

In [92]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ["J","I","H","G","F","E","D"]
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [94]:
num_pipeline=Pipeline(
    
    
    steps=[
        
        ('scaler', StandardScaler())
    ]
    
    
)

In [95]:
cat_pipeline=Pipeline(
    
    steps=[
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories]))
    ]
    
)

In [96]:
preprocessor=ColumnTransformer(
    [
        
        ('num_pipeline',num_pipeline,numerical_cols),
        ('cat_pipeline',cat_pipeline,categorical_cols)
    ]
)

In [98]:
preprocessor.fit_transform(X_train)
preprocessor.transform(X_test)

array([[-1.17935924,  0.24992847, -0.65471148, ...,  4.        ,
         3.        ,  6.        ],
       [-0.46105347, -1.22042647, -0.20613998, ...,  2.        ,
         4.        ,  5.        ],
       [-0.841333  ,  0.24992847, -1.10328299, ...,  4.        ,
         5.        ,  5.        ],
       ...,
       [-1.03147276, -2.62076452,  2.03671754, ...,  1.        ,
         3.        ,  6.        ],
       [ 0.91217813,  0.52999608, -1.55185449, ...,  4.        ,
         4.        ,  4.        ],
       [ 0.59527853,  1.0201144 ,  0.69100303, ...,  1.        ,
         0.        ,  1.        ]])

In [99]:
preprocessor.get_feature_names_out()

array(['num_pipeline__carat', 'num_pipeline__depth',
       'num_pipeline__table', 'num_pipeline__x', 'num_pipeline__y',
       'num_pipeline__z', 'cat_pipeline__cut', 'cat_pipeline__color',
       'cat_pipeline__clarity'], dtype=object)

In [100]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [101]:
X_train

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,2.560056,-2.550748,2.933861,2.229450,2.138209,1.738207,1.0,4.0,1.0
1,0.447392,-1.220426,1.139575,0.747550,0.656710,0.537733,2.0,5.0,1.0
2,0.637532,0.529996,0.242432,0.765404,0.700284,0.791951,3.0,2.0,3.0
3,1.482597,-0.170173,3.382432,1.318885,1.249310,1.243894,1.0,5.0,1.0
4,1.524851,0.249928,-0.206140,1.372448,1.380030,1.399249,2.0,3.0,4.0
...,...,...,...,...,...,...,...,...,...
43147,0.531899,0.459979,0.691003,0.667206,0.674140,0.721335,2.0,1.0,3.0
43148,-0.693447,-0.520257,-1.103283,-0.627225,-0.633065,-0.676864,4.0,6.0,4.0
43149,-0.989219,-1.010376,0.242432,-1.109289,-1.112374,-1.185299,2.0,4.0,7.0
43150,0.214999,0.740047,0.691003,0.354757,0.255834,0.396501,3.0,0.0,2.0


In [115]:
for i in range(len(list(models))):
    
    model=list(models.values())[i]
    pipe=Pipeline([
        ("model", model)
    ])
    pipe.fit(X_train,y_train)

    #Make Predictions
    y_pred=pipe.predict(X_test)

    #this is a validation(test) score
    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)

    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1224.5965418579544
MAE: 805.2743660189842
R2 score 90.56643685073512


Lasso
Model Training Performance
RMSE: 1224.6342579964453
MAE: 806.6592074144179
R2 score 90.5658557564055


Ridge
Model Training Performance
RMSE: 1224.5890370881857
MAE: 805.3678259425408
R2 score 90.56655247494308


Elasticnet
Model Training Performance
RMSE: 1559.9066553039286
MAE: 1052.1060818471647
R2 score 84.69311125672434


RandomForestRegressor
Model Training Performance
RMSE: 541.345238689499
MAE: 275.78942617055463
R2 score 98.15652035628437


